In [ ]:
class OptunaTuner:
    def __init__(self, model, params, X, y, splitter):
        self.model = model
        self.params = params
        self.X = X
        self.y = y
        self.splitter = splitter

    def objective(self, trial, X_train, y_train, X_val, y_val):
        param_values = {}
        for key, value_range in self.params.items():
            param_name = f"{key}_range"

            if value_range[0] <= value_range[1]:
                if isinstance(value_range[0], int) and isinstance(value_range[1], int):
                    param_values[param_name] = trial.suggest_int(param_name, value_range[0], value_range[1])
                else:
                    param_values[param_name] = trial.suggest_float(param_name, value_range[0], value_range[1])
            else:
                raise ValueError(f"Invalid range for {key}: low={value_range[0]}, high={value_range[1]}")

        self.set_model_hyperparameters(param_values)

        # Fit the model on the current training fold
        self.model.fit(X_train, y_train)

        # Get predicted probabilities for the validation fold
        y_probs = self.model.predict_proba(X_val)[:, 1]

        # Evaluate the model using AUC-ROC for the current fold
        auc_roc = roc_auc_score(y_val, y_probs)
        return auc_roc

    def set_model_hyperparameters(self, hyperparameters):
        if is_classifier(self.model):
            self.model.set_params(**hyperparameters)
        else:
            raise ValueError("Model does not support setting hyperparameters.")

    def tune(self, n_trials=100):
        study = optuna.create_study(direction="maximize")  # maximize AUC-ROC

        best_params_per_fold = []  # To store the best parameters for each fold

        # Iterate over each fold
        for X_train, X_val, y_train, y_val, val_index in tqdm(self.splitter.split_data(X=self.X, y=self.y)):
            objective_with_args = partial(self.objective, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)
            study.optimize(objective_with_args, n_trials=n_trials)

            # Store the best parameters for the current fold
            best_params_per_fold.append(study.best_params)

        # Find the most common parameters among the folds
        common_params = {}
        for param_name in self.params.keys():
            param_values = [params[param_name] for params in best_params_per_fold]
            most_common_value = max(set(param_values), key=param_values.count)
            common_params[param_name] = most_common_value

        print(f"Common parameters across folds: {common_params}")
        self.set_model_hyperparameters(common_params)
        return self.model